In [ ]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [ ]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
#print(mid.describe())
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
masses = [2,4,12,15,16,18,28,32,40,44]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,10:].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.iloc[:, idx].dropna().values
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

#### Vaclogger

In [ ]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

## Data processing

#### Comments

In [ ]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

#### CernOx temperature R-T conversion

In [ ]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [ ]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [ ]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 CU ESD sample halfway - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
""" 
#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)

 """
plt.savefig(r'./graphs/MID.png')
plt.show()


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 CU ESD sample halfway - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [ ]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 CU ESD sample halfway - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()